# Web Application
<img src="https://i.ibb.co/qYsF616/Screen-Shot-20220422155318.png" alt="Screen-Shot-20220422155318" border="0" />

In [2]:
#Mounting the gdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyngrok==4.1.1
!pip install flask_ngrok

  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15983 sha256=468a5743e20ee51e90a55e23c4eeb60007879af0ce4f2e6ddaca30a66c055afd
  Stored in directory: /root/.cache/pip/wheels/b1/d9/12/045a042fee3127dc40ba6f5df2798aa2df38c414bf533ca765
Successfully built pyngrok


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [5]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [6]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

In [7]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)


In [8]:
embed_dim = 256
latent_dim = 2048
num_heads = 8
vocab_size = 150000 
sequence_length = 20 
batch_size = 64

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

transformer.load_weights("/content/drive/MyDrive/Trained Model/For epoch 50-61% accuracy/check/weights-06-0.34.h5")

In [9]:
!cp "/content/drive/MyDrive/Trained Model/For epoch 30, 52% accuarcy/V_cmt.zip" "/content/"
!cp "/content/drive/MyDrive/Trained Model/For epoch 30, 52% accuarcy/V_code.zip" "/content/"

In [10]:
!unzip /content/V_cmt.zip
!unzip /content/V_code.zip

Archive:  /content/V_cmt.zip
   creating: content/vectorizer_cmt/
  inflating: content/vectorizer_cmt/keras_metadata.pb  
   creating: content/vectorizer_cmt/assets/
   creating: content/vectorizer_cmt/variables/
  inflating: content/vectorizer_cmt/variables/variables.data-00000-of-00001  
  inflating: content/vectorizer_cmt/variables/variables.index  
  inflating: content/vectorizer_cmt/saved_model.pb  
Archive:  /content/V_code.zip
   creating: content/vectorizer_code/
  inflating: content/vectorizer_code/keras_metadata.pb  
   creating: content/vectorizer_code/assets/
   creating: content/vectorizer_code/variables/
  inflating: content/vectorizer_code/variables/variables.data-00000-of-00001  
  inflating: content/vectorizer_code/variables/variables.index  
  inflating: content/vectorizer_code/saved_model.pb  


In [11]:
loaded_vectorize_layer_model_1 = tf.keras.models.load_model("/content/content/vectorizer_cmt")
cmt_vectorization = loaded_vectorize_layer_model_1.layers[0]

loaded_vectorize_layer_model_1 = tf.keras.models.load_model("/content/content/vectorizer_code")
code_vectorization = loaded_vectorize_layer_model_1.layers[0]

In [12]:
cmt_vocab = cmt_vectorization.get_vocabulary()
cmt_index_lookup = dict(zip(range(len(cmt_vocab)), cmt_vocab))
max_decoded_sentence_length = 20

In [13]:
def decode_sequence(input_sentence):
    input_sentence=input_sentence.lower()
    tokenized_input_sentence = code_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = cmt_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = cmt_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence

In [14]:
def final_clean_out(Comment):
  claen_cmt=[]
  cmt_lst=Comment.split(" ")
  cmt_lst=[i for i in cmt_lst if i !=""]
  for i in range(1,len(cmt_lst)):
    if cmt_lst[i] != cmt_lst[i-1]:
      claen_cmt.append(cmt_lst[i])
    if cmt_lst[i]=='end':
      break

  final_out=" ".join(claen_cmt[0:-1])
  return final_out

In [15]:
!ngrok authtoken 288k6I1toQej8k749xaaB83HdsQ_5Vgo9Jirc7zZw9PpPsRw5

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [16]:
cd /content/drive/MyDrive/Trained Model/Flask Project/Flask Project


/content/drive/MyDrive/Trained Model/Flask Project/Flask Project


In [17]:
ls

content/  static/  templates/  test1.py


In [ ]:
from flask_ngrok import run_with_ngrok 
from flask import Flask, request, render_template


app = Flask(__name__)
output = ""
entered = ""
run_with_ngrok(app)
@app.route("/submitbutton", methods =["POST"])
def submitButton():
    message = request.form['InsertCode']
    entered = message
    output1 = final_clean_out(decode_sequence(entered))
    output = output1
    return render_template("frontend.html", name=output, passed=entered)

@app.route("/generation", methods=["GET", "POST"])
@app.route('/')
def home():
        return render_template("frontend.html")

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a53a-35-194-138-7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [25/Apr/2022 16:08:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:27] "GET /static/styles/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [25/Apr/2022 16:08:28] "GET /static/styles/image.jpg HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:50] "GET /static/styles/styles.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:51] "GET /static/styles/image.jpg HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:08:52] "POST /submitbutton HTTP/1.1" 200 -
127.0.0.1 - - [25/Apr/2022 16:09:02] "GET /submitbutton HTTP/1.1" 405 -
